In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
% matplotlib inline

In [20]:
df = pd.read_csv("all_attributes.csv")
targets = df.iloc[:, [8, 9, 10, 11, 12]]
# independents = df.drop(df.columns[[0, 1, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14, 19]], axis=1)
# independents.head()
beer = pd.read_csv("anna.csv")
independents = pd.concat([pd.read_csv("tfidfsentiment.csv"), pd.read_csv("sentiments.csv")], axis=1)

In [21]:
independents['stopwordcount'] = beer['stopwordcount']
independents['charcount'] = beer['charcount']
independents['cursewordcount'] = beer['cursewordcount']
independents['uniquecursewordcount'] = beer['uniquecursewordcount']

In [22]:
independents.head()

,Unnamed: 0,exceptional,excellent,fantastic,wonderful,highly,perfect,delicious,incredible,amazing,...,ok,average,bad,corn,sipper,group_lda,stopwordcount,charcount,cursewordcount,uniquecursewordcount
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,42,415,0,0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,1,42,450,0,0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,21,312,0,0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,27,312,0,0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,49,383,0,0


In [23]:
targets.head()

,review/appearance,review/aroma,review/overall,review/palate,review/taste
0,4.0,4.0,4.0,4.0,4.0
1,4.0,3.5,3.5,3.5,3.0
2,3.5,4.0,3.5,3.5,3.5
3,3.0,3.0,2.5,3.0,3.0
4,4.0,3.0,3.0,3.5,2.5


#### Apearance

In [24]:
X = independents
y = targets.iloc[:, 0]

regr = linear_model.LinearRegression().fit(X, y)
print("Regular R^2: " + str(regr.score(X, y)))
print("Regular SSE: " + str(sum((regr.predict(X) - y) ** 2)))

# las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
# print("Lasso R^2: " + str(las_reg.score(X, y)))
# print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Regular R^2: 0.325055523498
Regular SSE: 8769.351027828938


#### Aroma

In [25]:
X = independents
y = targets.iloc[:, 1]

regr = linear_model.LinearRegression().fit(X, y)
print("Regular R^2: " + str(regr.score(X, y)))
print("Regular SSE: " + str(sum((regr.predict(X) - y) ** 2)))

# las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
# print("Lasso R^2: " + str(las_reg.score(X, y)))
# print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Regular R^2: 0.44519910807
Regular SSE: 9639.817852948026


#### Palate

In [26]:
X = independents
y = targets.iloc[:, 3]

regr = linear_model.LinearRegression().fit(X, y)
print("Regular R^2: " + str(regr.score(X, y)))
print("Regular SSE: " + str(sum((regr.predict(X) - y) ** 2)))

# las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
# print("Lasso R^2: " + str(las_reg.score(X, y)))
# print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Regular R^2: 0.432616774205
Regular SSE: 9491.299370216564


#### Taste

In [27]:
X = independents
y = targets.iloc[:, 4]

regr = linear_model.LinearRegression().fit(X, y)
print("Regular R^2: " + str(regr.score(X, y)))
print("Regular SSE: " + str(sum((regr.predict(X) - y) ** 2)))

# las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
# print("Lasso R^2: " + str(las_reg.score(X, y)))
# print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Regular R^2: 0.511282810183
Regular SSE: 9405.363478217347


#### Overall

In [28]:
X = independents
y = targets.iloc[:, 2]

regr = linear_model.LinearRegression().fit(X, y)
print("Regular R^2: " + str(regr.score(X, y)))
print("Regular SSE: " + str(sum((regr.predict(X) - y) ** 2)))

# las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
# print("Lasso R^2: " + str(las_reg.score(X, y)))
# print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Regular R^2: 0.443074418684
Regular SSE: 10242.71280194145
